In [2]:
import pandas as pd
import talib

# Load CSV
df = pd.read_csv("/home/iatell/projects/meta-learning/data/Bitcoin_BTCUSDT_kaggle_1D_candles.csv")

# Ensure proper column names (adjust if your file differs)
# Expected: timestamp, open, high, low, close, volume
df.columns = [col.lower() for col in df.columns]

# Dictionary of candlestick recognition functions from TA-Lib
candle_funcs = {
    'CDLDOJI': talib.CDLDOJI,
    'CDLHAMMER': talib.CDLHAMMER,
    'CDLHANGINGMAN': talib.CDLHANGINGMAN,
    'CDLENGULFING': talib.CDLENGULFING,
    'CDLSHOOTINGSTAR': talib.CDLSHOOTINGSTAR,
    'CDLMORNINGSTAR': talib.CDLMORNINGSTAR,
    'CDLEVENINGSTAR': talib.CDLEVENINGSTAR,
    # add more as needed...
}

# Apply candlestick functions
labels = pd.Series([0] * len(df), index=df.index, dtype=int)

for name, func in candle_funcs.items():
    pattern_result = func(df['open'], df['high'], df['low'], df['close'])
    # TA-Lib returns +100/-100/0, so keep whichever is nonzero
    labels = labels.where(pattern_result == 0, pattern_result)

# Build final result
result = pd.DataFrame({
    "timestamp": df["timestamp"],
    "label": labels
})

print(result.head())


    timestamp  label
0  2018-01-01      0
1  2018-01-02      0
2  2018-01-03      0
3  2018-01-04      0
4  2018-01-05      0
